In [ ]:
import os
import requests
import time
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display

In [ ]:

load_dotenv('../.env')
API_KEY = os.getenv('PUBLIC_API_KEY')

BASE_URL = f'http://openapi.seoul.go.kr:8088/{API_KEY}/json/tbLnOpendataRtmsV'

years = [2023, 2024, 2025]
districts = {
    '서초구': '11650',
    '강남구': '11680',
    '송파구': '11710'
}

rows_all = []
step = 1000

for year in years:
    for gu_nm, gu_cd in districts.items():
        start = 1

        while True:
            end = start + step - 1
            url = f'{BASE_URL}/{start}/{end}/{year}/{gu_cd}'

            res = requests.get(url)
            if res.status_code != 200:
                print('요청 실패:', year, gu_nm)
                break

            data = res.json().get('tbLnOpendataRtmsV', {})
            rows = data.get('row', [])

            if not rows:
                break

            rows_all.extend(rows)
            print(f'{year} {gu_nm} {start}~{end} 수집')

            start += step
            time.sleep(2)

df = pd.DataFrame(rows_all)

if df.empty:
    print("데이터가 없습니다.")
else:
    df = df[df['BLDG_USG'] == '아파트']

    df = df[
        [
            'RCPT_YR',        # 접수연도
            'CGG_CD',         # 자치구 코드
            'CGG_NM',         # 자치구명
            'STDG_CD',        # 법정동 코드
            'STDG_NM',        # 법정동 명
            'BLDG_NM',        # 건물명
            'CTRT_DAY',       # 계약일
            'THING_AMT',	  # 물건금액(만원)
            'ARCH_AREA',	  # 건물면적(㎡)
            'LAND_AREA',	  # 토지면적(㎡)
            'FLR',	          # 층
            'ARCH_YR',        # 건축년도
            'BLDG_USG',       # 건물용도
        ]
    ]

    # 계약일자 기준 필터링 추가
    # CTRT_DAY를 datetime으로 변환
    # 거래는 2022-12-31에 했으나 신고를 2023년에 하는 케이스가 존재
    df['CTRT_DAY'] = pd.to_datetime(df['CTRT_DAY'], format='%Y%m%d', errors='coerce')
    
    # 2023-01-01 ~ 2025-12-31 사이만 필터링
    df = df[
        (df['CTRT_DAY'] >= '2023-01-01') & 
        (df['CTRT_DAY'] <= '2025-12-31')
    ]

    # 중복 제거
    print(f"\n중복 제거 전: {len(df):,}건")
    
    df_clean = df.drop_duplicates(subset=[
        'CTRT_DAY',
        'BLDG_NM', 
        'ARCH_AREA',
        'FLR',
        'THING_AMT'
    ], keep='first')

    print(f"중복 제거 후: {len(df_clean):,}건")

    # CTRT_DAY를 다시 문자열로 변환 (저장 시 형식 유지)
    df_clean['CTRT_DAY'] = df_clean['CTRT_DAY'].dt.strftime('%Y%m%d')

    df_clean.to_csv(
        '../data/seoul_apartment_2023_2025_gangnam.csv',
        index=False,
        encoding='utf-8-sig'
    )

    display(df.head())

In [ ]:
# 저장된 csv 읽기
df = pd.read_csv('../data/seoul_apartment_2023_2025_gangnam.csv', encoding='utf-8-sig')

In [24]:
# 중복값 확인 (거래일 + 건물명 + 면적 + 층 + 가격)
duplication = df[df.duplicated(subset=[
    'CTRT_DAY',  # 계약일
    'BLDG_NM',   # 건물명
    'ARCH_AREA', # 건물면적(㎡)
    'LAND_AREA', # 토지면적(㎡)
    'FLR',       # 층
    'THING_AMT'  # 물건금액(만원)
], keep='first')]

print(f"중복 건수: {len(duplication)}건")
display(duplication.head())

중복 건수: 0건


,RCPT_YR,CGG_CD,CGG_NM,STDG_CD,STDG_NM,BLDG_NM,CTRT_DAY,THING_AMT,ARCH_AREA,LAND_AREA,FLR,ARCH_YR,BLDG_USG


In [27]:
# 컬럼 정보
display(df.info())

# 상위 5개 행 확인
display(df.head())

# 결측치 확인
display(df.isna().sum()) # 전체 확인

# 요약 통계
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28309 entries, 0 to 28308
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   RCPT_YR    28309 non-null  int64  
 1   CGG_CD     28309 non-null  int64  
 2   CGG_NM     28309 non-null  object 
 3   STDG_CD    28309 non-null  int64  
 4   STDG_NM    28309 non-null  object 
 5   BLDG_NM    28309 non-null  object 
 6   CTRT_DAY   28309 non-null  int64  
 7   THING_AMT  28309 non-null  int64  
 8   ARCH_AREA  28309 non-null  float64
 9   LAND_AREA  28309 non-null  float64
 10  FLR        28309 non-null  float64
 11  ARCH_YR    28309 non-null  int64  
 12  BLDG_USG   28309 non-null  object 
dtypes: float64(3), int64(6), object(4)
memory usage: 2.8+ MB


None

,RCPT_YR,CGG_CD,CGG_NM,STDG_CD,STDG_NM,BLDG_NM,CTRT_DAY,THING_AMT,ARCH_AREA,LAND_AREA,FLR,ARCH_YR,BLDG_USG
0,2023,11650,서초구,10800,서초동,서초1차e-편한세상,20231228,220000,130.53,0.0,4.0,2004,아파트
1,2023,11650,서초구,10700,반포동,반포파크빌,20231227,175000,110.79,0.0,4.0,2002,아파트
2,2023,11650,서초구,10300,우면동,서초힐스,20231227,123000,74.97,0.0,8.0,2012,아파트
3,2023,11650,서초구,10100,방배동,SK리더스뷰(파스텔시티),20231226,140000,84.95,0.0,13.0,2006,아파트
4,2023,11650,서초구,10100,방배동,롯데캐슬포레스트,20231222,450000,239.33,0.0,1.0,2003,아파트


RCPT_YR      0
CGG_CD       0
CGG_NM       0
STDG_CD      0
STDG_NM      0
BLDG_NM      0
CTRT_DAY     0
THING_AMT    0
ARCH_AREA    0
LAND_AREA    0
FLR          0
ARCH_YR      0
BLDG_USG     0
dtype: int64

,RCPT_YR,CGG_CD,STDG_CD,CTRT_DAY,THING_AMT,ARCH_AREA,LAND_AREA,FLR,ARCH_YR
count,28309.000000,28309.000000,28309.000000,2.830900e+04,2.830900e+04,28309.000000,28309.0,28309.000000,28309.000000
mean,2024.191918,11685.177859,10687.929634,2.024240e+07,2.214347e+05,86.568668,0.0,10.441520,1977.011869
std,0.778861,23.913459,454.510684,7.772980e+03,1.281218e+05,34.596883,0.0,7.357358,221.554687
min,2023.000000,11650.000000,10100.000000,2.023010e+07,1.100000e+04,12.100000,0.0,-1.000000,0.000000
25%,2024.000000,11680.000000,10300.000000,2.024020e+07,1.345000e+05,59.960000,0.0,5.000000,1992.000000
50%,2024.000000,11680.000000,10700.000000,2.024083e+07,1.986000e+05,84.810000,0.0,9.000000,2004.000000
75%,2025.000000,11710.000000,10900.000000,2.025032e+07,2.770000e+05,99.920000,0.0,14.000000,2012.000000
max,2025.000000,11710.000000,11800.000000,2.025123e+07,1.900000e+06,301.470000,0.0,68.000000,2025.000000


In [29]:
# 데이터 전처리
print("="*80)
print("Step 1: 데이터 전처리")
print("="*80)
print(f"\n원본 데이터: {len(df):,}건")

# 날짜 변환
df['CTRT_DAY'] = pd.to_datetime(df['CTRT_DAY'], format='%Y%m%d')

# 년월 컬럼 생성
df['년월'] = df['CTRT_DAY'].dt.to_period('M')


Step 1: 데이터 전처리

원본 데이터: 28,309건


In [ ]:
print("="*80)
print("데이터 전처리")
print("="*80)
print(f"\n원본 데이터: {len(df):,}건")

# 날짜 변환 -> 월별 그룹화를 위해서
df['CTRT_DAY'] = pd.to_datetime(df['CTRT_DAY'], format='%Y%m%d')

# yearMonth 컬럼 생성 -> 월별 추이를 위해
df['yearMonth'] = df['CTRT_DAY'].dt.to_period('M')

# 4. 규모 분류 함수 -> 일일이 면적 비교는 비효율적으로 규모 분류
def classify_size(area):
    """
    전용면적 기준 규모 분류
    - 소형: 60㎡ 이하
    - 중소형: 60~85㎡
    - 중형: 85~102㎡
    - 중대형: 102~135㎡
    - 대형: 135㎡ 초과
    """
    if pd.isna(area):
        return '알수없음'
    elif area <= 60:
        return '소형'
    elif area <= 85:
        return '중소형'
    elif area <= 102:
        return '중형'
    elif area <= 135:
        return '중대형'
    else:
        return '대형'

# 5. size 컬럼 추가
df['size'] = df['ARCH_AREA'].apply(classify_size)

print("\n규모별 분포:")
print(df['size'].value_counts().sort_index())

print("\n구별 분포:")
print(df['CGG_NM'].value_counts())

print("\n기간:")
print(f"시작: {df['CTRT_DAY'].min()}")
print(f"종료: {df['CTRT_DAY'].max()}")
print(f"총 {df['년월'].nunique()}개월")

# 6. 기본 통계
print("\n거래금액 기본 통계:")
print(df['THING_AMT'].describe())

print("\n✅ Step 1 완료!")

데이터 전처리

원본 데이터: 28,309건

규모별 분포:
규모
대형      2516
소형      8267
중대형     4019
중소형    12058
중형      1449
Name: count, dtype: int64

구별 분포:
CGG_NM
송파구    11858
강남구     9479
서초구     6972
Name: count, dtype: int64

기간:
시작: 2023-01-01 00:00:00
종료: 2025-12-31 00:00:00
총 36개월

거래금액 기본 통계:
count    2.830900e+04
mean     2.214347e+05
std      1.281218e+05
min      1.100000e+04
25%      1.345000e+05
50%      1.986000e+05
75%      2.770000e+05
max      1.900000e+06
Name: THING_AMT, dtype: float64

✅ Step 1 완료!
